# Cell Types Ontology to Neo4J

## Context

This notebook has been used to display the Cell Types Ontology together with relevant Brain Region entities in Neo4J Bloom

## Imports

In [ ]:
import rdflib
from rdflib import OWL, RDFS, SKOS, RDF, Namespace
import re
import getpass
from bmo.neo4j import ontology_to_neo4j

## Prerequisite

You have to download the [Cell Types Ontology](https://webprotege.kcp.bbp.epfl.ch/#projects/968c9144-bca3-4436-bdb5-6529d46016b9/edit/Classes) as well as the [Brain Region Ontology](https://webprotege.kcp.bbp.epfl.ch/#projects/968c9144-bca3-4436-bdb5-6529d46016b9/edit/Classes) from WebProtégé in TTL format. Please contact DKE should you need to open a WebProtégé account.

## Setup

In [ ]:
base = "" # Provide the directory into which you have downloaded the Cell Types and Brain Region Ontology from WebProtégé

In [ ]:
uri = "bolt://localhost:7687"
username = "neo4j"

In [ ]:
password = getpass.getpass()

## Load the ontologies

In [ ]:
cell_type_ontology = rdflib.Graph()
cell_type_ontology.parse(f"{base}/celltypes.ttl")

In [ ]:
brain_region_ontology = rdflib.Graph()
brain_region_ontology.parse(f"{base}/brainregion.ttl")

## Get required brain regions

Since brain regions are part of the *Brain Region Ontology* and not the *Cell Type Ontology* we will be adding the necessary triples from the Brain Region Ontology to the ontology to be displayed in Neo4J

In [ ]:
brain_regions = list()
for s, p, o in cell_type_ontology.triples((None, None, rdflib.term.URIRef("https://bbp.epfl.ch/ontologies/core/bmo/canHaveBrainRegion"))):
    for s2, p2, o2 in cell_type_ontology.triples((s, OWL.someValuesFrom, None)):
        brain_regions.append(o2)

In [ ]:
brain_regions = list(set(brain_regions))

In [ ]:
for b in brain_regions:
    for el in [rdflib.term.URIRef(RDFS.label), rdflib.term.URIRef(SKOS.notation), rdflib.term.URIRef(SKOS.prefLabel), rdflib.term.URIRef(RDFS.subClassOf)]: # [rdflib.term.URIRef(RDFS.label), rdflib.term.URIRef(SKOS.notation), rdflib.term.URIRef(SKOS.prefLabel), rdflib.term.URIRef(RDFS.subClassOf)]
        for s, p, o in brain_region_ontology.triples((b, el, None)):
            cell_type_ontology.add((s, p, o))
cell_type_ontology.add((rdflib.term.URIRef("https://neuroshapes.org/BrainRegion"), RDFS.label, rdflib.term.Literal("Brain Region")))
cell_type_ontology.add((rdflib.term.URIRef("https://neuroshapes.org/BrainRegion"), RDF.type, OWL.Class))

## Serialize the ontology to TTL format

In [ ]:
cell_type_ontology.serialize(destination=f"{base}/ontology.ttl")

## Get the ontology as database into Neo4J

In [ ]:
ontology_to_neo4j(
    uri, username, password, f"{base}/ontology.ttl", format="turtle")